# Scraping StreetEasy's historical data

StreetEasy (a brand and registered trademark of Zillow, Inc.) is New York City's leading real estate marketplace. 
The website displys a current rental (and sales) listings only - and does not allow access to past listings which are now off the market. 
And so, to get data on the website's past condition We will use the **'wayback machine'** to access a 'screenshot' at a past data.

### We will focus on **Manhatten**

Our process will be:

- For each **month** in the years **2014-2019**:
    - Search in the 'wayback machine' (a 'internetarchives.com' project) under the website: http://streeteasy.com/for-rent/manhattan.
    - For each **page** in the first **10** pages:
        - List all **links** to rentel listings on page
        - For each **link** on a **page**:
            - scrape info and append into mysql Dataset

***

### Import libraries

General

In [1]:
# for printing current time when crashes and waits
import time
import datetime

# for scraping htmls
from bs4 import BeautifulSoup

# to avoid getting blocked
import random

import requests  # for soup

import pandas as pd # for dataFrames

import re # for regex search
import lxml # better then html detection 
import cchardet # to assit in charecter detection

Webdriver specific

In [2]:
from selenium import webdriver

# in order to not expose our password we will use 'getpass'
import getpass
# To connect to mysql use:
from sqlalchemy import create_engine

# for opening the web browser

PATH = "C:\\Program Files (x86)\\chromedriver.exe" 
# -----------------------------> NOTICE: 1. did you download chromedriver? 2. is the location correct?
driver = webdriver.Chrome(PATH)

# for loading the page : start only after loading
from selenium.webdriver.chrome.options import Options

***

# Functions

## Utility functions:

TAKES a url for a webpage and RETURNS a list of links (for each rental proposal) on that page 

In [3]:
def outisde_scrape_links(url):  
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')  
        
    page_link_lst = []
    for link in soup.findAll("div", {"class": "details-title"}) :
        page_link_lst.append("https://web.archive.org"+link.find("a")['href']) # find the link attached 
    return(page_link_lst)

TAKES a url of a single rental proposal including detaild information and RETURNS a single row'de dataFrame

In [4]:
def inside_scrape(url):
    print(url,'\n') # see current url proccesed 

    r = requests.get(url) 
    soup = BeautifulSoup(r.text, 'lxml') # read as lxml for faster encoding 
    
    # create an empty data Frame
    # ----------------------------------------

    df_single_search = pd.DataFrame()
    
    # error handling - initialize columns as nulls
    
    scrape_year, scrape_day, scrape_month, scrape_hour,  = 'NaN','NaN','NaN','NaN'
    description_short, description_long = 'NaN','NaN'
    price, rooms,amenities = 'NaN','NaN','NaN'
    zip_code, address = 'NaN','NaN' 
    days_on_market, price_history, lister = 'NaN','NaN','NaN'
    price_at_point, last_price_change ='NaN','NaN'

    # insert data to dataFrame
    # ----------------------------------------
    
    # 1. Price, Short description, Room description:
    details = soup.find("div", {"class": "details"})
   
    if details:
        # short description of the listing
        details_describe = details.find_all('span', attrs={'class':re.compile(r'detail_cell')})
        if details_describe:
            description_short = "".join(re.findall(r'\>(.*?)\<' , str(details_describe))) # DESC_SHORT > to file
        #print(description_short)
         # description of rooms
        details_a = details.find_all('span', attrs={'class':'nobreak'})
        if details_a:
            rooms = "".join(re.findall(r'\>(.*?)\<' , str(details_a))) # ROOMS > to df
    # the price asked:
        price = re.findall(r'(?=\$).+', str(details))[0]  # RENT PRICE > to df
    
    df_single_search['price'] = [price] # notice passing as list 
    df_single_search['des_short'] = [description_short] # notice passing as list 
    df_single_search['des_rooms'] = [rooms] # notice passing as list 
    

    # 2. Detailed description (by owner):
    describe_soup = soup.find("blockquote", {"class": "description_togglable hidden"})
    if describe_soup:
        description_long = describe_soup.get_text() # DESC_LONG > to file
    df_single_search['des_long'] = [description_long] # notice passing as list 

    # 3. Amenities included:
    amenities_soup = soup.find('div', attrs={'class':re.compile(r'amenities')})
    if amenities_soup:
        amenities = amenities_soup.get_text().replace("Amenities", "").replace("\n\n", " ")
    df_single_search['amenities'] = [amenities] # notice passing as list 

    # 4. Zip code, Address:
    address = soup.select_one('h2 ~ p')
    if address:
        address = address.get_text()
    # seperate address, city, zipcode
        address_dirty = "".join(re.findall(r'(?!  ).?', ''.join(address.split('\n'))))
        if address_dirty:
            address_dirty = address_dirty.split('  ')[0].replace(u'\xa0', u' ').split('  ') # the encoding ('utf-8') casuses space to be written as '\xa0'
            if len(address_dirty) == 2:
        # zip code
                if address_dirty[1]:
                    zip_code = re.findall(r'[0-9]*$', address_dirty[1].strip())[0]
        # address
                if address_dirty[0]:
                    address = address_dirty[0]
    else: # if the address is not specified - use the ad title
        address = soup.find('h1', attrs={'class':'building-title nd___highlighted'})
        if address:
            address = address.get_text()
    
    df_single_search['zip'] = [zip_code] # notice passing as list 
    df_single_search['address'] = [address] # notice passing as list 
    
    # 4. Days on the market, Last price change:
    facts_dirty = soup.select('h6 ~ *')
    if facts_dirty:
        for i in facts_dirty:
        # Days On Market
            #print(i.get_text(), "\n")
            #print("end\n")
            if "days on StreetEasy" in i.get_text():
                days_on_market = i.get_text().strip()

        # Last Price Change
            if "days ago" in i.get_text():
                last_price_change = i.get_text().strip()
                last_price_change = " ".join([i.strip() for i in last_price_change.split('\n')])
                
    df_single_search['last_price_change'] = [last_price_change] # notice passing as list 
    df_single_search['days_on_market'] = [days_on_market] # notice passing as list 


    # 5. Price history (date), Lister's (names), Price at date:
    price_change_dirty = soup.select('h2 ~ table')
     # Price change history
    if price_change_dirty:
        history_table = pd.read_html(str(price_change_dirty))[0].fillna(method='ffill', axis=0) # save into a pandas DataSet - fill NaN!
        if history_table.shape[1] == 3:
            print("\nhistory_table: \n",history_table)
            
            if not history_table.iloc[:,0].empty:
                price_history = "|".join(list(history_table.iloc[:,0])) # notice - before added 'str' line
                #print("\nprice_history: ", price_history)

            if not history_table.iloc[:,1].empty:
                lister = "|".join(list(history_table.iloc[:,1]))
                #print("\nlister: ", lister)

            if not history_table.iloc[:,2].empty:
                price_at_point = "|".join(list(history_table.iloc[:,2]))
    df_single_search['price_history'] = price_history
    df_single_search['lister'] = lister
    df_single_search['price_at_point'] = price_at_point
    
    # 7. when was the page archived - Year, Day, Month, Hour:
    archive_dates = re.findall(r'(?<=FILE ARCHIVED ON ).*(?=AND)' , str(soup))
    if archive_dates:
        archive_dates = archive_dates[0]
        print(archive_dates,"\n")
        scrape_hour = re.findall(r'\d\d:\d\d:\d\d' , archive_dates)[0].strip()
        scrape_year =  re.findall(r'\d{4}.*$' , archive_dates)[0].strip()
        scrape_day =  re.findall(r'\d+(?=,)' , archive_dates)[0].strip()
        scrape_month =  re.findall(r'(?<=\d\d:\d\d:\d\d).+?(?= )' , archive_dates)[0].strip()
    
    df_single_search['scrape_year'] = scrape_year
    df_single_search['scrape_day'] = scrape_day
    df_single_search['scrape_month'] = scrape_month
    df_single_search['scrape_hour'] = scrape_hour

    print(df_single_search) # show what was scraped
    return(df_single_search)

TAKES a url for a rental proposal, opens it and checks if the word *Hrm.* exist in the first h2 object (meaning the page *isn't* archived). Returns True / False

In [5]:
def check_archived(web_page):
    requests_session = requests.Session()
    try:
        r = requests_session.get(web_page, timeout=10.0)
        soup = BeautifulSoup(r.text, 'lxml') 
        top_h2 = soup.select_one('h2')
        if top_h2:
            top_h2 = top_h2.get_text()
            print (top_h2)
            if 'Hrm.' in top_h2:
                print("Page not archived")
                return (False)
            else:
                print("Page archived")
                return (True)
    except requests.Timeout as err:
        print(err)
        return(False)
    except requests.RequestException as err:
        print(err)
        return(False)
    return(False)

Takes a url for the first page and RETURNS the number pages that existed (even though we will only scrape 10)

In [6]:
def check_last_page_nbr(first_page):
    r = requests.get(first_page)
    soup = BeautifulSoup(r.text, 'html.parser')  
    last_page_num = soup.select('a:nth-last-child(2)')[-2].get_text()
    return (int(last_page_num))

## MAIN FUNCTION

TAKES the title page of a of the zone (e.g Manhattan) in a specific date (e.g jan 2014) and RETURNS a dataFrame containting the rent listings from the first **10**  pages.

In [7]:
def scrape_moment(first_page, tmp_df_name):
    current_page_url = first_page # on the first round - use first page as current
    
    # create an empty dataFrame
    dfObj = pd.DataFrame(columns=['scrape_day','scrape_month','scrape_year',
                                  'scrape_hour','des_short', 'des_long', 'price', 'des_rooms', 'amenities',
                                  'zip', 'address', 'days_on_market', 'price_history', 'lister', 'price_at_point',
                                  'last_price_change'])
    
    #last_page_num = check_last_page_nbr(first_page) # how many pages existed in total (even though we only scrape the first 10)
    #last_page_num = 'NaN'
    #dfObj['page_pop_size'] = last_page_num

    scrape_page = current_page_url # on the first round - use first page as current
    page_list = list(range(2,11)) # scrape only 9 pages (after the first page)

    pages_success_scrape = set() # create an empty set to hold the number of pages that were succesfully scraped
    print (page_list)
    page_num = 1 
    while page_num < 10: # while there's pages left to scrape from the list (we remove a single page each round)
        page_link_lst = outisde_scrape_links(current_page_url) # get links on current page
        for link in page_link_lst:
            time.sleep(random.randint(10, 20)) # avoid getting blocked
            # check if the page was archived
            if check_archived(link):
                dfObj = dfObj.append(inside_scrape(link)) 
                pages_success_scrape.add(scrape_page) # add to set
            print("Scraped: ",page_num, " out of : 10") # to see where we are in the scrape

        scrape_page = random.choice(page_list) # select a random page number value from the list of posible pages
        print("\nLooking into: ",scrape_page)
        page_list.remove(scrape_page) # take out p. number of the current scraped page from the list of possible pages

        dfObj.to_sql(tmp_df_name, engine, index=False, if_exists='append') # push to sql
        print("\n", current_page_url,"\n")
        page_num = page_num+1 # for counting pages scraped
        #current_page_url = str(first_page)+"?page={}".format(page_num)# change to the next page
        current_page_url = str(first_page)+"?page={}".format(scrape_page)# change to the next page
        
        print("\nafter:", current_page_url,"\n")
        print("\nScraped the following pages: ",pages_success_scrape)
        print("\nWhich makes a total of :",len(pages_success_scrape))
        time.sleep(random.randint(10, 120)) # avoid getting blocked

    return (dfObj)

## Connect to my local SQL server

In [8]:
hostname = "localhost"
username = "root"
password = getpass.getpass()
database = "karta"
engine = create_engine(f"mysql+pymysql://{username}:{password}@{hostname}/{database}")

 ·············


The dataFrame that we use will have the following columns:

In [9]:
base_df = pd.DataFrame(columns=['scrape_day','scrape_month','scrape_year','scrape_hour','des_short',
                                'des_long', 'price', 'des_rooms', 'amenities', 'zip', 'address',
                                'days_on_market', 'price_history', 'lister', 'price_at_point','last_price_change'])

# Run

Define the years and months we're intrested in

In [14]:
year_lst = ['2019']
#month_lst = ['04','05','06','07','08','09','10','11','12']
month_lst = ['01','02','03','04','05','06','07','08','09','10','11','12']

Run the code for each year and month

In [15]:
for year in year_lst:
    time.sleep(random.randint(180, 300)) # avoid getting blocked
    for month in month_lst:
        time.sleep(random.randint(180, 300)) # avoid getting blocked
        print(year) # see on which year we're on
        print(month) # see on which month we're on
        # clear / create dataFrame (in sql):
        tmp_df_name = f'ann_rent_{year}_manhattan_{month}'
        base_df.to_sql(tmp_df_name, engine, index=False, if_exists='replace')  
        # change url according to month and year
        url = f'https://web.archive.org/web/{year}{month}/http://streeteasy.com/for-rent/manhattan'
        print(f'{url}\n')
        #push current month (and year) to sql
        scrape_moment(url, tmp_df_name)

2019
01
https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan

[2, 3, 4, 5, 6, 7, 8, 9, 10]

Looking into:  6

 https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan 


after: https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan?page=6 


Scraped the following pages:  set()

Which makes a total of : 0

Looking into:  7

 https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan?page=6 


after: https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan?page=7 


Scraped the following pages:  set()

Which makes a total of : 0

Looking into:  2

 https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan?page=7 


after: https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan?page=2 


Scraped the following pages:  set()

Which makes a total of : 0

Looking into:  10

 https://web.archive.org/web/201901/http://streeteasy.com/for-rent/manhattan?page=2 